oad the previously downloaded Citybik.es dataset as a Python dictionary. You can make use of the
json module presented. You can find the full documentation for the json module here. After the
dictionary is loaded, explore its contents from an interactive shell

In [3]:
import json
with open("to-bike.json") as f:
    dataset = json.load(f)
    

In [4]:
dataset.keys()

dict_keys(['network'])

In [5]:
type(dataset["network"])

dict

In [6]:
dataset["network"].keys()

dict_keys(['id', 'name', 'location', 'href', 'company', 'system', 'source', 'stations'])

In [7]:


type(dataset["network"]["stations"])



list

In [17]:
dataset["network"]["stations"][5]

{'id': '8c12fcd7311c98e9a948af38caa4d051',
 'name': '01. Certosa / P.le Avis',
 'latitude': 45.084126990755045,
 'longitude': 7.575631141662598,
 'timestamp': '2024-10-27T04:10:47.675366Z',
 'free_bikes': 5,
 'empty_slots': 3,
 'extra': {'uid': '529',
  'number': 1,
  'status': 'offline',
  'score': 3.0,
  'reviews': 28}}

Count and print the number of active stations (a station is active if its extra.status field is "online")

In [22]:
all_stations = dataset["network"]["stations"]
active_stations = [ station for station in all_stations if station["extra"]["status"] == "online" ]
print("Number of active stations", len(active_stations))

Number of active stations 1


Count and print the total number of bikes available (field free_bikes) and the number of free docks
(field empty_slots) throughout all stations.

In [23]:
bikes_avail = sum([station["free_bikes"] for station in all_stations])
free_docks = sum([ station["empty_slots"] for station in all_stations])
print("Bikes available", bikes_avail)
print("Free docks", free_docks)

Bikes available 95
Free docks 57


(*) Given the coordinates (latitude, longitude) of a point (e.g. 45.074512, 7.694419), identify the
closest bike station to it that has available bikes. For computing the distance among two points
(given their coordinates), you can use the function distance_coords() defined in the code snippet
below (which is an implementation of the great-circle distance):

In [27]:
from math import cos, acos, sin

def distance_coords(lat1, lng1, lat2, lng2):
    deg2rad = lambda x: x * 3.141592 / 180
    lat1, lng1, lat2, lng2 = map(deg2rad, [ lat1, lng1, lat2, lng2 ])
    R = 6378100 # Radius of the Earth, in meters
    return R * acos(sin(lat1) * sin(lat2) + cos(lat1) * cos(lat2) * cos(lng1 - lng2))

In [38]:
lat2 = 45.074512
lng2 = 7.694419
for i in range(len(all_stations)):
    current_station = all_stations[i]
    lat,long = current_station["latitude"],current_station["longitude"]
    distance = distance_coords(lat,long,lat2,lng2)
    if i == 0 and station["free_bikes"] > 0:
        closest_station = current_station
        min_distance = distance
    else:
        if (distance < min_distance and station["free_bikes"] > 0):
            min_distance = distance
            closest_station = current_station
            
print("closest station is: "+ closest_station["name"])        
print(min_distance)    

closest station is: 10. Gallo Praile
6128.518994474587


In [33]:
def distance_from_point(dataset, lat, lng):
    closest = (None, None)
    for station in all_stations:
        closest_station, closest_distance = closest
        current_distance = distance_coords(lat,lng,station["latitude"], station["longitude"])

        if station["free_bikes"] > 0 and (closest_distance is None or current_distance < closest_distance):
            closest = (station, current_distance)
    return closest

station, distance = distance_from_point(dataset, 45.074512, 7.694419)
print("Closest station:", station["name"])
print("Distance:", distance, "meters")

Closest station: 09. Rigola
Distance: 6219.58278998704 meters


In [40]:
max(all_stations, key=lambda station: station["free_bikes"])


{'id': '0df8838da4cf2ffe0eaf28629b902dd7',
 'name': '01. Municipio',
 'latitude': 45.094383006473215,
 'longitude': 7.524325847625732,
 'timestamp': '2024-10-27T04:10:47.783380Z',
 'free_bikes': 5,
 'empty_slots': 3,
 'extra': {'uid': '594',
  'number': 1,
  'status': 'offline',
  'score': 2.1,
  'reviews': 16}}

In [44]:
def distance_from_point_pro(dataset,lat,lng):
    tup = [(station, distance_coords(lat,lng,station["latitude"], station["longitude"])) for station in all_stations if station["free_bikes"] > 0]
    return min(tup , key = lambda tup: tup[1])

station, distance = distance_from_point_pro(dataset, 45.074512,7.694419)
print("Closest station:", station["name"])
print("Distance:", distance, "meters")
print("Number of available bikes:", station["free_bikes"])

Closest station: 09. Rigola
Distance: 6219.58278998704 meters
Number of available bikes: 5
